In [ ]:
%load_ext autoreload
%autoreload 2

#%% Change working directory from the workspace root to the ipynb file location. Turn this addition off with the DataSciece.changeDirOnImportExport setting
import os
try:
    os.chdir(r'C:\Users\kevin\Documents\Workspace\psylit-experiments')
    print("Changed cwd:", os.getcwd())
except:
    print("cwd:", os.getcwd())

from collections import *
from statistics import mean
from functools import *
from os import linesep as EOL
from itertools import *
import re
from tic import preprocess

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import networkx as nx
plt.rcParams['figure.figsize'] = 16,10
np.random.seed(0)


In [ ]:
import networkx

import flair

from flair.embeddings import FlairEmbeddings, BertEmbeddings
from flair.data import Sentence

# init Flair embeddings
#flair_forward_embedding = FlairEmbeddings('multi-forward')
#flair_backward_embedding = FlairEmbeddings('multi-backward')

# init multilingual BERT
bert_embedding = BertEmbeddings('bert-base-cased')

from flair.embeddings import StackedEmbeddings


In [ ]:
data_root = r'..\datasets\2_txtalb_Novel450'
txt=preprocess.read_pg(data_root + r'\EN_1818_Shelley,Mary_Frankenstein_Novel.txt')
print(len(txt), 'chars')

In [ ]:
from segtok.segmenter import split_single
raw_sentences = list(split_single(txt))

def parse_sentences(raw_sentences):
    return [Sentence(s, use_tokenizer=True) for s in raw_sentences]

sentences = parse_sentences(raw_sentences)
print(len(sentences), 'sentences')

In [ ]:
from itertools import islice

def sliding_window(length, max_win_length=1, stride=1):
    i = 0
    while i < length:
        yield slice(i, i+max_win_length)
        i += stride

In [13]:
import time

def batch_embed(sentences, windows, embedding=None):
    embedding = embedding or BertEmbeddings('bert-base-cased')
    windows = list(windows)
    n = len(windows)
    for i, win in enumerate(windows):
        print("Batch", i+1, "/", n)
        ss = sentences[win]
        d = timed(lambda: embedding.embed(ss), fmt="\t{d:.5f}s")
    return sentences

def np_embeddings(sentences):
    return np.array([t.embedding.numpy() for s in sentences for t in s])

def timed(f, fmt="{d:.5f}s"):
    t0 = time.perf_counter()
    res = f()
    d = time.perf_counter() - t0
    print(fmt.format(d=d))
    return res

In [14]:
n = 10
ss = timed(lambda: batch_embed(parse_sentences(raw_sentences[:n]), sliding_window(n, 1, 1)), fmt="Total: {d:.5f}s")
e1 = np_embeddings(ss)

2019-05-31 12:02:49,760 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
Batch 1 / 10
	0.12553s
Batch 2 / 10
	0.13499s
Batch 3 / 10
	0.21045s
Batch 4 / 10
	0.10032s
Batch 5 / 10
	0.10391s
Batch 6 / 10
	0.10540s
Batch 7 / 10
	0.17494s
Batch 8 / 10
	0.35992s
Batch 9 / 10
	0.22165s
Batch 10 / 10
	0.29149s
Total: 13.72474s


In [ ]:
ss = timed(lambda: batch_embed(parse_sentences(raw_sentences[:n]), sliding_window(n, 6, 3)))
e2 = np_embeddings(ss)

In [ ]:
ss = timed(lambda: batch_embed(parse_sentences(raw_sentences[:n]), sliding_window(n, n, n)))
e3 = np_embeddings(ss)

In [ ]:
del ss
e1.shape, e2.shape, e3.shape

In [ ]:
err1 = np.linalg.norm(e3 - e1, axis=1)
err2 = np.linalg.norm(e3 - e2, axis=1)

In [ ]:
sqerr1 = np.power(err1, 2)
sqerr2 = np.power(err2, 2)
plt.plot(sqerr1)
plt.plot(sqerr2)
plt.ylabel("Squared error")
plt.xlabel("Token")
plt.legend(['win 1, stride 1', 'win 6, stride 3'])
plt.show()

In [ ]:
sample = [t.text for s in sentences for t in s][200:220]
print(" ".join(sample))

In [15]:
ss = timed(lambda: batch_embed(sentences, sliding_window(len(sentences), 6, 3)), fmt="Total: {d:.5f}s")
E = np_embeddings(ss)

2019-05-31 12:03:20,944 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
Batch 1 / 1134
	0.00003s
Batch 2 / 1134
	0.00011s
Batch 3 / 1134
	0.00019s
Batch 4 / 1134
	0.00010s
Batch 5 / 1134
	0.00011s
Batch 6 / 1134
	0.00013s
Batch 7 / 1134
	0.00008s
Batch 8 / 1134
	0.00013s
Batch 9 / 1134
	0.00002s
Batch 10 / 1134
	0.00005s
Batch 11 / 1134
	0.00007s
Batch 12 / 1134
	0.00007s
Batch 13 / 1134
	0.00011s
Batch 14 / 1134
	0.00012s
Batch 15 / 1134
	0.00012s
Batch 16 / 1134
	0.00009s
Batch 17 / 1134
	0.00005s
Batch 18 / 1134
	0.00008s
Batch 19 / 1134
	0.00008s
Batch 20 / 1134
	0.00008s
Batch 21 / 1134
	0.00007s
Batch 22 / 1134
	0.00007s
Batch 23 / 1134
	0.00006s
Batch 24 / 1134
	0.00042s
Batch 25 / 1134
	0.00005s
Batch 26 / 1134
	0.00005s
Batch 27 / 1134
	0.00003s
Batch 28 / 1134
	0.00005s
Batch 29 / 1134
	0.00008s
Batch 30 / 1134
	0.00008s
Batch 31 / 1134
	

Batch 300 / 1134
	0.72959s
Batch 301 / 1134
	1.71622s
Batch 302 / 1134
	1.47188s
Batch 303 / 1134
	1.08472s
Batch 304 / 1134
	0.95016s
Batch 305 / 1134
	0.72585s
Batch 306 / 1134
	0.73410s
Batch 307 / 1134
	0.91012s
Batch 308 / 1134
	0.91375s
Batch 309 / 1134
	0.82560s
Batch 310 / 1134
	0.88026s
Batch 311 / 1134
	0.62419s
Batch 312 / 1134
	0.66992s
Batch 313 / 1134
	0.66129s
Batch 314 / 1134
	0.57533s
Batch 315 / 1134
	0.72083s
Batch 316 / 1134
	0.99561s
Batch 317 / 1134
	0.98462s
Batch 318 / 1134
	1.00237s
Batch 319 / 1134
	1.52356s
Batch 320 / 1134
	0.85551s
Batch 321 / 1134
	0.51585s
Batch 322 / 1134
	0.78975s
Batch 323 / 1134
	0.84858s
Batch 324 / 1134
	0.97205s
Batch 325 / 1134
	1.07908s
Batch 326 / 1134
	1.14216s
Batch 327 / 1134
	1.57940s
Batch 328 / 1134
	1.57192s
Batch 329 / 1134
	0.85829s
Batch 330 / 1134
	1.47737s
Batch 331 / 1134
	1.44861s
Batch 332 / 1134
	0.76885s
Batch 333 / 1134
	1.43571s
Batch 334 / 1134
	1.17280s
Batch 335 / 1134
	0.95229s
Batch 336 / 1134
	1.38952s
B

Batch 603 / 1134
	1.53197s
Batch 604 / 1134
	1.48588s
Batch 605 / 1134
	0.88459s
Batch 606 / 1134
	0.88903s
Batch 607 / 1134
	0.96447s
Batch 608 / 1134
	0.98856s
Batch 609 / 1134
	0.91727s
Batch 610 / 1134
	1.00197s
Batch 611 / 1134
	1.45962s
Batch 612 / 1134
	1.11804s
Batch 613 / 1134
	0.67378s
Batch 614 / 1134
	0.60129s
Batch 615 / 1134
	0.29412s
Batch 616 / 1134
	0.68329s
Batch 617 / 1134
	0.96059s
Batch 618 / 1134
	0.95288s
Batch 619 / 1134
	1.04555s
Batch 620 / 1134
	1.00821s
Batch 621 / 1134
	1.11018s
Batch 622 / 1134
	1.17583s
Batch 623 / 1134
	0.77903s
Batch 624 / 1134
	1.26862s
Batch 625 / 1134
	1.32346s
Batch 626 / 1134
	0.70898s
Batch 627 / 1134
	1.63174s
Batch 628 / 1134
	1.32294s
Batch 629 / 1134
	0.71397s
Batch 630 / 1134
	0.84681s
Batch 631 / 1134
	1.72672s
Batch 632 / 1134
	1.71683s
Batch 633 / 1134
	0.81987s
Batch 634 / 1134
	1.00682s
Batch 635 / 1134
	1.02183s
Batch 636 / 1134
	0.69542s
Batch 637 / 1134
	1.18759s
Batch 638 / 1134
	1.15150s
Batch 639 / 1134
	0.76880s
B

Batch 906 / 1134
	1.52735s
Batch 907 / 1134
	2.04046s
Batch 908 / 1134
	2.05211s
Batch 909 / 1134
	1.85942s
Batch 910 / 1134
	0.60333s
Batch 911 / 1134
	1.59986s
Batch 912 / 1134
	1.59552s
Batch 913 / 1134
	1.66061s
Batch 914 / 1134
	1.75145s
Batch 915 / 1134
	0.79145s
Batch 916 / 1134
	0.71854s
Batch 917 / 1134
	1.11316s
Batch 918 / 1134
	0.77006s
Batch 919 / 1134
	0.73536s
Batch 920 / 1134
	1.49295s
Batch 921 / 1134
	1.43711s
Batch 922 / 1134
	0.71745s
Batch 923 / 1134
	0.99789s
Batch 924 / 1134
	1.09492s
Batch 925 / 1134
	1.17790s
Batch 926 / 1134
	1.09168s
Batch 927 / 1134
	0.96216s
Batch 928 / 1134
	0.90733s
Batch 929 / 1134
	1.34815s
Batch 930 / 1134
	1.27237s
Batch 931 / 1134
	2.13692s
Batch 932 / 1134
	1.78238s
Batch 933 / 1134
	0.97706s
Batch 934 / 1134
	0.96036s
Batch 935 / 1134
	0.85017s
Batch 936 / 1134
	0.79330s
Batch 937 / 1134
	0.79803s
Batch 938 / 1134
	0.87239s
Batch 939 / 1134
	0.95660s
Batch 940 / 1134
	0.96087s
Batch 941 / 1134
	1.19201s
Batch 942 / 1134
	1.16883s
B

In [16]:
E.shape

(87648, 3072)

In [20]:
with open(r'outputs\frankenstein_tokens.txt', 'w') as f:
    i = 0
    for s in ss:
        for t in s:
            f.write("\t".join([str(i),t.text]) + "\n")
            i += 1
np.save(r'outputs\frankenstein_bert', E)

In [17]:
from sys import getsizeof
locs = pd.DataFrame([{'var':k, 's':getsizeof(v)} for k,v in locals().items()])
#locs.rename(index=str, columns={0:'var', 1:'size'})
locs.sort_values(by='s',ascending=False,inplace=True)
locs

,s,var
130,1077018736,E
128,2089072,e1
105,846496,txt
108,30712,raw_sentences
127,30120,ss
110,30120,sentences
99,1464,BertEmbeddings
31,1056,ChainMap
27,1056,UserList
26,1056,UserDict


In [ ]:
bert_embedding

In [ ]:
E.shape